In [26]:
#connect to s3 with the sdk
from boto.s3.connection import S3Connection
from PIL import Image 
import webbrowser
import datetime
import boto3
from boto.s3.key import Key

from matplotlib.pyplot import imshow
import numpy as np
from PIL import Image
client = boto3.client('rekognition', 'us-east-1')
%matplotlib inline

#You must change these to your own password and bucket information.
conn = S3Connection()

from matplotlib.pyplot import imshow
import json
from pprint import pprint

%matplotlib inline
import matplotlib
matplotlib.rc("savefig", dpi=100)
import cv2
from collections import namedtuple



print ('done')

done


In [27]:
#print the bucket name and contents
mybuckets = conn.get_all_buckets()
print(len(mybuckets))

print ('print info about this bucket')
#this bucket is read-public, so feel free to follow this example code without changing 
#anything besides public/private key
#bucketname='rekognitionbotoartifact' #interacting with this public bucket
#Using my own bucket
bucketname='dhouli48'
s3bucket = conn.get_bucket(bucketname)
print(s3bucket)
size = 0
count = 0
for key in s3bucket.list():
    size += key.size
    count += 1
 
print ("%.3f GB" % (size*1.0/1024/1024/1024))
print ("num files: "+str(count))

for key in s3bucket.list():
        print ("{name}\t{size}\t{modified}".format(
                name = key.name,
                size = key.size,
                modified = key.last_modified,
                ))

2
print info about this bucket
<Bucket: dhouli48>
0.001 GB
num files: 5
Adam Sandler wig.png	150125	2017-09-15T12:54:15.000Z
Alexandra Daddario.png	97285	2017-09-15T12:51:26.000Z
Dwayne Johnson.png	76546	2017-09-15T12:51:26.000Z
Noel Catti.png	403914	2017-09-15T12:51:26.000Z
sandler.png	478960	2017-09-15T12:46:53.000Z


In [28]:
#a demo of looping over a s3 bucket. printing out the MTCNN and the num_faces 
#from rekognition's api. Looking at printed output gives interesting results about errors
#coming from both sides. Errors are few though, and are usually worth taking in exchange
#for the cost savings
import commands

#looping over s3bucket, printing out recognizing celebs.  
#Determine 3 things
#1. Is there a celeb in the photo
#2. The Confidence level 
#3. The celeb name
        
for key in s3bucket.list():
    #use this line to control how many loops
    if count==50:
        break
    count += 1
    if ((count % 100) == 0):
        print ('at count: '+str(count)+'\n')
        
    print(key.name)
    try:
        #Call the recognition that it is a celeb
        response = client.recognize_celebrities(
            Image={
                'S3Object': {
                'Bucket': bucketname,
                'Name': key.name
                #,'Version': '1'
            }
        }
        )
        
        
        #detect if celeb
        num_api_faces=len(response['CelebrityFaces'])
        output= str(response)
        print('Rekognition celebrities detected count: '+str(num_api_faces))
        #print(response)
        print("=============================")
       
    except:
        print ('Could not find item: '+key.name)
        print('')
        failcount=(failcount+1)
        

        #display the confidence level if celeb
    for celeb_face in response['CelebrityFaces']:
        dmsg = "Confidence level this is celeb: {MatchConfidence}%".format(MatchConfidence=celeb_face['MatchConfidence'])
        print(dmsg)
        print("=============================")
        
        #display confidence level if not celeb
    for n_celeb_face in response['UnrecognizedFaces']:
        pmsg = "Confidence level this is not celeb: {Confidence}%".format(Confidence=n_celeb_face['Confidence'])
        print(pmsg)
        print("=============================")
        
    
    #determine name
    #Note there were special characters after some names so needed a try/catch
    try:
        for name in response['CelebrityFaces']:
            msg = "this celeb is {Name}".format(Name=name['Name'])
            print(msg)
            print("=============================")
            print("=============================")
    except:
        print("Unable to determine Celeb name")
        print("=============================")

Adam Sandler wig.png
Rekognition celebrities detected count: 0
Confidence level this is not celeb: 99.9626083374%
Alexandra Daddario.png
Rekognition celebrities detected count: 1
Confidence level this is celeb: 100.0%
this celeb is Alexandra Daddario
Dwayne Johnson.png
Rekognition celebrities detected count: 1
Confidence level this is celeb: 100.0%
this celeb is Dwayne Johnson
Noel Catti.png
Rekognition celebrities detected count: 0
Confidence level this is not celeb: 99.9747848511%
sandler.png
Rekognition celebrities detected count: 1
Confidence level this is celeb: 100.0%
this celeb is Adam Sandler


In [29]:
#copy file to my git directory
!cp /home/ubuntu/MTCNN-face-filter/demo/MTCNN_Face_Filter_for_Rekognition_Data_Prep.ipynb /home/ubuntu/git/
#then manually push from git dir